In [10]:
import os
import re
import numpy as np
from PIL import Image

import tensorflow as tf
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Conv2D, Convolution2D, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
import ipyplot
import matplotlib.pyplot as plt

os.chdir('c:/code/jupyter-notebook/meta-learning/siamese')

def read_image(filename, byteorder='>'):
    image = Image.open(filename).convert('L')
    return image
    # with open(filename, 'rb') as f:
    #     buffer = f.read()
    #
    # header, width, height, maxval = re.search(
    #     b"(^P5\s(?:\s*#.*[\r\n])*"
    #     b"(\d+)\s(?:\s*#.*[\r\n])*"
    #     b"(\d+)\s(?:\s*#.*[\r\n])*"
    #     b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    #
    # return np.frombuffer(
    #     buffer,
    #     dtype='u1' if int(maxval) < 256 else byteorder + 'u2',
    #     count=int(width) * int(height),
    #     offset=len(header)
    # ).reshape((int(height), int(width)))

In [11]:
image = read_image('./data/orl_faces/s41/7.pgm')
print(np.array(image).shape)

(2560, 1920)


In [12]:
size = 2
total_sample_size = 10000


def get_data(size, total_sample_size):
    #read the image
    image = np.array(read_image('data/orl_faces/s1/1.pgm', 'rw+'))
    image = image[::size, ::size]
    dims1 = image.shape[0]
    dims2 = image.shape[1]

    count = 0

    genuine_x = np.zeros([total_sample_size, 2, dims1, dims2, 1])
    genuine_y = np.zeros([total_sample_size, 1])

    for i in range(42):
        for j in range(int(total_sample_size/42)):
            idx1 = idx2 = 0

            while idx1 == idx2:
                idx1 = np.random.randint(10)
                idx2 = np.random.randint(10)

            image1 = read_image(f'data/orl_faces/s{i+1}/{idx1+1}.pgm', 'rw+')
            image2 = read_image(f'data/orl_faces/s{i+1}/{idx2+1}.pgm', 'rw+')

            image1 = np.array(image1.resize((46, 56)))
            image2 = np.array(image2.resize((46, 56)))

            genuine_x[count, 0, :, :, 0] = image1
            genuine_x[count, 1, :, :, 0] = image2

            genuine_y[count] = 1

            count += 1

    count = 0

    imposter_x = np.zeros([total_sample_size, 2, dims1, dims2, 1])
    imposter_y = np.zeros([total_sample_size, 1])

    for i in range(int(total_sample_size/10)):
        for j in range(10):
            idx1 = idx2 = 0

            while idx1 == idx2:
                idx1 = np.random.randint(42)
                idx2 = np.random.randint(42)

            image1 = read_image(f'data/orl_faces/s{idx1+1}/{j+1}.pgm', 'rw+')
            image2 = read_image(f'data/orl_faces/s{idx2+1}/{j+1}.pgm', 'rw+')

            image1 = np.array(image1.resize((46, 56)))
            image2 = np.array(image2.resize((46, 56)))

            imposter_x[count, 0, :, :, 0] = image1
            imposter_x[count, 1, :, :, 0] = image2

            imposter_y[count] = 0

            count += 1

    X = np.concatenate([genuine_x, imposter_x], axis=0)/255
    Y = np.concatenate([genuine_y, imposter_y], axis=0)

    return X, Y

In [13]:
X, Y = get_data(size, total_sample_size)

print(X.shape)
print(Y.shape)

(20000, 2, 56, 46, 1)
(20000, 1)


In [14]:
x_train, x_text, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [15]:
def build_network(input_shape):
    seq = Sequential()

    nb_filter = [6, 12]
    kernel_size = 3

    seq.add(Conv2D(nb_filter[0], (kernel_size, kernel_size),
                   input_shape=input_shape, padding='valid', data_format='channels_last'))
    seq.add(Activation('relu'))
    seq.add(MaxPool2D(pool_size=(2,2)))
    seq.add(Dropout(.25))

    seq.add(Conv2D(nb_filter[1], (kernel_size, kernel_size),
                   input_shape=input_shape, padding='valid', data_format='channels_last'))
    seq.add(Activation('relu'))
    seq.add(MaxPool2D(pool_size=(2,2)))
    seq.add(Dropout(.25))

    seq.add(Flatten())
    seq.add(Dense(128, 'relu'))
    seq.add(Dropout(.1))
    seq.add(Dense(50, 'relu'))

    return seq

In [16]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

base_network = build_network(input_dim)
network_a = base_network(img_a)
network_b = base_network(img_b)

In [17]:
def euclidean_distance(vects):
    vect1, vect2 = vects
    return K.sqrt(K.sum(K.square(vect1 - vect2), axis=1, keepdims=True))

def euclid_shape(shape):
    shape1, shape2 = shape
    return (shape1[0], 1)

distance = Lambda(euclidean_distance, euclid_shape)([network_a, network_b])

In [18]:
epochs = 13
rms = RMSprop()

model = Model(inputs=[img_a, img_b], outputs=distance)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1- y_true)*K.square(K.maximum(margin - y_pred, 0)))

model.compile(loss=contrastive_loss, optimizer=rms)

In [19]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1]

tensorboard = TensorBoard(log_dir="log/{}".format(time()))
model.fit([img_1, img_2], y_train, validation_split=.25,
          batch_size=128, verbose=2, epochs=epochs, callbacks=[tensorboard])

Epoch 1/13
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
88/88 - 29s - loss: 0.1879 - val_loss: 0.2543
Epoch 2/13
88/88 - 19s - loss: 0.1168 - val_loss: 0.2060
Epoch 3/13
88/88 - 19s - loss: 0.0845 - val_loss: 0.1003
Epoch 4/13
88/88 - 18s - loss: 0.0650 - val_loss: 0.0713
Epoch 5/13
88/88 - 20s - loss: 0.0544 - val_loss: 0.0513
Epoch 6/13
88/88 - 19s - loss: 0.0468 - val_loss: 0.0459
Epoch 7/13
88/88 - 18s - loss: 0.0424 - val_loss: 0.0431
Epoch 8/13
88/88 - 18s - loss: 0.0383 - val_loss: 0.0329
Epoch 9/13
88/88 - 17s - loss: 0.0348 - val_loss: 0.0275
Epoch 10/13
88/88 - 19s - loss: 0.0320 - val_loss: 0.0224
Epoch 11/13
88/88 - 18s - loss: 0.0298 - val_loss: 0.0183
Epoch 12/13
88/88 - 17s - loss: 0.0275 - val_loss: 0.0210
Epoch 13/13
88/88 - 18s - loss: 0.0264 - val_loss: 0.0161


In [36]:
pred = model.predict([x_text[:, 0], x_text[:, 1]])

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

compute_accuracy(pred, y_test)

0.9689655172413794

In [21]:
def compare_face(face_1, face_2):
    size = 2
    idx1 = idx2 = 0
    while idx1==idx2:
        idx1 = np.random.randint(10)
        idx2 = np.random.randint(10)

    img_1 = read_image(f'data/orl_faces/s{face_1+1}/{idx1+1}.pgm')
    img_2 = read_image(f'data/orl_faces/s{face_2+1}/{idx2+1}.pgm')

    img_1 = img_1.resize((46, 56))
    img_2 = img_2.resize((46, 56))

    img_1.show()
    img_2.show()

    img_1 = np.array(img_1)
    img_2 = np.array(img_2)

    dim1, dim2 = img_1.shape
    test_data = np.zeros([1, 2, dim1, dim2, 1])
    test_y = np.zeros([1, 1])

    test_data[0, 0, :, :, 0] = img_1
    test_data[0, 1, :, :, 0] = img_2
    test_y[0, :] = face_1 == face_2

    pred = model.predict([test_data[:, 0], test_data[:,1]])

    print(pred)
    print(idx1, idx2)

In [35]:
compare_face(40, 40)

[[16.283669]]
4 0
